In [1]:
import pandas as pd
import numpy as np
import math 
import os 
from datetime import datetime, timedelta
import pymap3d # python3 -m pip install pymap3d
import random
import pickle
from scipy.special import lambertw
from scipy import stats
from scipy.interpolate import interp1d as inte1
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d
output_notebook() 


save_data_bool = False # Save data or not


Loading BokehJS ...

In [2]:
#Verify sol_file and Hp before run
dataset="cabspotting" # OPTIONS ARE: cabspotting privamov
user="oilrag" # Examples: abboip oilrag 51 90 14
Hp=4 #same as offline

local_url="datasets/"+dataset+"-tree/"+user+".csv"
sol_file=user+'/databuild_Hp'+str(Hp)
exists = os.path.isfile(local_url)

if exists:
    data = pd.read_csv(local_url,names=["latitude","longitude","timestamp"])
    print("uploaded from local file")
else:
    print("data not found")

if not os.path.exists(sol_file):
    os.makedirs(sol_file)

data['time']=[datetime.utcfromtimestamp(tstamp/1000) for tstamp in data['timestamp']]
data['elapsedtime']=(data['timestamp']-data['timestamp'][0])/1000
whole_duration = data['elapsedtime'][len(data['timestamp'])-1]/60
print('Whole duration: '+str(math.floor(whole_duration))+' min')    
    
data=data.set_index('time').sort_index().copy()
ell_grs80 = pymap3d.Ellipsoid('grs80') 
data['altitude']=np.zeros(len(data.index))
lat0, lon0, h0 = data['latitude'][0],data['longitude'][0],data['altitude'][0]
data['x'], data['y'], data['u_enu'] = pymap3d.geodetic2enu(data['latitude'], data['longitude'], data['altitude'], lat0, lon0, h0, ell=ell_grs80)
print(data.head())  

uploaded from local file
Whole duration: 1126 min
                     latitude  longitude      timestamp  elapsedtime  \
time                                                                   
2008-05-17 10:00:06  37.74574 -122.42685  1211018406000          0.0   
2008-05-17 10:01:17  37.74967 -122.42729  1211018477000         71.0   
2008-05-17 10:03:13  37.75233 -122.43419  1211018593000        187.0   
2008-05-17 10:04:03  37.75696 -122.43463  1211018643000        237.0   
2008-05-17 10:05:13  37.76122 -122.43501  1211018713000        307.0   

                     altitude           x            y     u_enu  
time                                                              
2008-05-17 10:00:06       0.0    0.000000     0.000000  0.000000  
2008-05-17 10:01:17       0.0  -38.777282   436.197503 -0.015077  
2008-05-17 10:03:13       0.0 -646.852405   731.460882 -0.074827  
2008-05-17 10:04:03       0.0 -685.585574  1245.356220 -0.158740  
2008-05-17 10:05:13       0.0 -719.030524  

In [3]:
# Real transmitted data
#oilrag 20000-30000
#abboip 50000-60000
#51 158250-188250
Tmin = 20000
Tmax = 30000
# Tmax = data['elapsedtime'][-1]
xr = data['x'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
yr = data['y'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
ur = data['u_enu'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
Timer = data['elapsedtime'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
print('transmitted positions',len(Timer))
# Resampling the data with sample time ts, averaging over the intervals
ts = 30  # Sampling time
tmin = Timer[0]
tmax = Timer[-1]  # Final time
nt = int((tmax-tmin)/ts)+1
print('resampling points',nt)

newx = []
newy = []
newu = []
newb = []
newTimer = []
newvel = []
xtmin1 =0
ytmin1 =0
tmin1 =-100000
for i in range(0,nt):
    xt = data['x'][(data['elapsedtime'] >= tmin+i*ts) & (data['elapsedtime'] < tmin+(i+1)*ts)].mean() 
    yt = data['y'][(data['elapsedtime'] >= tmin+i*ts) & (data['elapsedtime'] < tmin+(i+1)*ts)].mean() 
    ut = data['u_enu'][(data['elapsedtime'] >= tmin+i*ts) & (data['elapsedtime'] <tmin+(i+1)*ts)].mean() 
    if np.isnan(xt) == False:
        newx += [xt]
        newy += [yt]
        newu += [ut]
        newb += [1]
        newvel += [np.sqrt((xt-xtmin1)**2+(yt-ytmin1)**2)/((i+1)*ts-tmin1)]
        xtmin1 =xt
        ytmin1 =yt
        tmin1 =(i+1)*ts
    else:  # Fill with zeros when no transmitted
        newx += [0]
        newy += [0]
        newu += [0]
        newb += [0]
        newvel += [-1]
    newTimer += [tmin+(i+1)*ts]

xs = np.array(newx)
ys = np.array(newy)
bs = np.array(newb)
us = np.array(newu)
vs = np.array(newvel)
Time = np.array(newTimer)

transmitted positions 152
resampling points 331


In [4]:
save_data_raw = pd.DataFrame()
save_data_raw['Timer'] = Timer
save_data_raw['xr'] = xr
save_data_raw['yr'] = yr
pickle.dump( save_data_raw, open( sol_file+"/save_data_raw_online.pkl", "wb" ) )

save_data = pd.DataFrame()
save_data['Time'] = Time
save_data['xs'] = xs
save_data['ys'] = ys
save_data['us'] = us
save_data['bs'] = bs
save_data['vs_m_s'] = vs
save_data['vs_km_h'] = vs*3.6
pickle.dump( save_data, open( sol_file+"/save_data_online.pkl", "wb" ) )

save_data_raw = pickle.load( open( sol_file+"/save_data_raw_online.pkl", "rb" ) )
save_data = pickle.load( open( sol_file+"/save_data_online.pkl", "rb" ) )

# Vector for plotting, removed the non-transmitted instants
Timep = Time[bs != 0]
xsp = xs[bs != 0] 
ysp = ys[bs != 0] 

fig = figure(plot_width=900, plot_height=450)
fig.line(Timer, xr, line_color='navy', legend_label="x", alpha=0.2)
fig.circle(Timer, xr, line_color='navy', legend_label="x", fill_alpha=0.2, alpha=0.2)
fig.line(Timep, xsp, line_color='navy', legend_label="x")
fig.circle(Timep, xsp, line_color='navy', color = 'cyan', legend_label="x")
fig.line(Timer, yr, line_color='red', legend_label="y", alpha=0.2)
fig.circle(Timer, yr, line_color='red', legend_label="y", fill_alpha=0.2, alpha=0.2)
fig.line(Timep, ysp, line_color='red', legend_label="y")
fig.circle(Timep, ysp, line_color='red', color = 'pink', legend_label="y")
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "y"
show(fig)

fig = figure(plot_width=900, plot_height=450)
fig.line(xsp, ysp, line_color='navy', legend_label="p")
fig.circle(xsp, ysp, color='navy', legend_label="p")
fig.line(xr, yr, line_color='navy', legend_label="p raw", alpha=0.2)
fig.circle(xr, yr, color='navy', legend_label="p raw", fill_alpha=0.2, alpha=0.2)
fig.xaxis.axis_label = "x"
fig.yaxis.axis_label = "y"
show(fig)

## Implementation

In [5]:
# Create the dynamic system state
n = 30
X = np.array([0]*n)
Y = np.array([0]*n)
N = np.array([0]*n)

system = pd.DataFrame()
system['X'] = X
system['Y'] = Y
system['N'] = N

def system_update(system,x,y,b):
    dict = {'X': x, 'Y': y, 'N': b}
    system = system.append(dict, ignore_index = True).drop(index=[0]).reset_index(drop=True)
    return system

def privacy(system):
    n = system['N'].sum()
    if n == 0:
        barx = 0
        bary = 0
        priv = 0
    else:
        barx = np.dot(system['X'], system['N'])/n
        bary = np.dot(system['Y'], system['N'])/n
        #priv = np.dot(((system['X'] - barx)**2 + (system['Y'] - bary)**2), system['N'])/n 
        priv = np.dot((system['X'] - barx)**2 + (system['Y'] - bary)**2, system['N'])/n 
    return barx, bary, priv

def utility(x,y):
    return np.sqrt(x**2 + y**2)

def reset_system(system):
    n = len(system)
    X = np.array([0]*n)
    Y = np.array([0]*n)
    N = np.array([0]*n)

    system['X'] = X
    system['Y'] = Y
    system['N'] = N
    return system

from sys import path
from casadi import *
from pylab import *

nbuf = n
# Control
u = MX.sym("u")
v = MX.sym("v")
b = MX.sym("b")

# State
x = MX.sym("x",nbuf)
y = MX.sym("y",nbuf)
nx = MX.sym("nx",nbuf)

# Matrices
A = np.zeros((nbuf, nbuf))
B = np.zeros((nbuf, 1))
for i in range(nbuf-1):
    A[i,i+1] = 1

B[nbuf-1,0] = 1
A.tolist()
B.tolist()
A = DM(A)
B = DM(B)

# Dynamics
xplus = A@x + B@u
yplus = A@y + B@v
nplus = A@nx + B@b

# Discrete time dynamics function
F = Function('F', [x,u],[xplus])

#Privacity function
O = DM(np.ones((1,nbuf)))
xbar = (O@(x*nx))/((O@nx)+1e-8)
ybar = O@(y*nx)/((O@nx)+1e-8)
P = (O@( ((x-xbar)**2 + (y-ybar)**2)*nx) )/((O@nx)+1e-10)
J = Function('J', [x,y,nx],[xbar, ybar, P],['x','y','nx'],['xbar', 'ybar','P'])

# Test privacy functions
x1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
y1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
nx1 = [1]*nbuf

Jk = J(x=x1,y=y1,nx=nx1)
print(Jk)
system2 = pd.DataFrame()
system2['X'] = x1
system2['Y'] = y1
system2['N'] = nx1

print(privacy(system2))

{'P': DM(0.31646), 'xbar': DM(-0.00456867), 'ybar': DM(0.0240504)}
(-0.004568665549133005, 0.024050441656272438, 0.3164599990716751)


In [6]:
system = reset_system(system)
barxk = []
baryk = []
privk = []

util = []
for i in range(0,nt,1):
    bi = bs[i] 
    # Update the real system
    system = system_update(system,xs[i],ys[i],bi)
    barx, bary, priv = privacy(system)
    barxk += [barx] 
    baryk += [bary] 
    privk += [priv] 


Data_real = pd.DataFrame()
Data_real['x'] = xs
Data_real['y'] = ys
Data_real['b'] = bs
Data_real['barx'] = barxk
Data_real['bary'] = baryk
Data_real['priv'] = privk
Data_real['u_enu'] = us

pickle.dump( Data_real, open( sol_file+"/Data_real_online.pkl", "wb" ) )

# GEO-I

In [7]:
import random
def GeoI_LPPM(x,y,epsilon):
    theta_GeoI = random.uniform(0,2*math.pi)
    r_GeoI = -1/epsilon*(np.real(lambertw((random.random()-1)/math.exp(1),k=-1))+1);
    x_ctrl = x + r_GeoI*math.cos(theta_GeoI)
    y_ctrl = y + r_GeoI*math.sin(theta_GeoI)
    return x_ctrl, y_ctrl, r_GeoI 

In [8]:
x_obf=np.zeros_like(xs)
y_obf=np.zeros_like(ys)
time_obf=np.zeros_like(ys)
for i in range(nt):
    time_0= time.time()
    x_obf[i] ,y_obf[i], r =GeoI_LPPM(xs[i],ys[i],0.00275)
    time_obf[i]=time.time()-time_0
    
delta=np.zeros((2,nt))
delta[0,]=x_obf-xs
delta[1,]=y_obf-ys


system_obf = reset_system(system)
barxk_obf = []
baryk_obf = []
privk_obf = []

util = []
for i in range(0,nt,1):
    bi = bs[i] 
    # Update the real system
    # Update the obfuscated system
    system_obf = system_update(system_obf,x_obf[i],y_obf[i],bi)
    barx_obf, bary_obf, priv_obf = privacy(system_obf)
    barxk_obf += [barx_obf] 
    baryk_obf += [bary_obf] 
    privk_obf += [priv_obf] 
    # Compute utility
    util += [utility(delta[0,i],delta[1,i])*bi]



Data_obf = pd.DataFrame()
Data_obf['x'] = x_obf
Data_obf['y'] = y_obf
Data_obf['U'] = delta[0,]
Data_obf['V'] = delta[1,]
Data_obf['barx'] = barxk_obf
Data_obf['bary'] = baryk_obf
Data_obf['priv'] = privk_obf
Data_obf['util'] = util
Data_obf['u_enu'] = us
Data_obf['runtime'] = time_obf

Data_obf.to_csv(sol_file+'GeoI.csv')
    

## Worst privacy case method

In [9]:
## Solve new method
            #mode=0 => Using 2 canditates
            #mode=1 => MPC horizon=1
            #mode=2 => Take 360 candidates, one for each 1°

def solve_method(xs,ys,bs,vs,horizon,nbuf,p_bound,data_offline,mode):
    nt = len(xs)
    x_sol = []
    y_sol = []
    U_sol = []
    V_sol = []
    barx_sol    = []
    bary_sol    = []
    priv_sol    = []
    util_sol    = []
    time_sol    = []
    tra_sol     = []
    util_sol_b  = []
    dife        = []
    sol_min     = [] #index of 8's solutions proposed in mode 0
    #X0 = np.array(xs[0:nbuf]).tolist()
    #Y0 = np.array(ys[0:nbuf]).tolist()
    #N0 = np.array(bs[0:nbuf]).tolist()
    X0 = np.array([0]*nbuf).tolist()
    Y0 = np.array([0]*nbuf).tolist()
    N0 = np.array([0]*nbuf).tolist()

    system_opt = pd.DataFrame()
    system_opt['X'] = X0
    system_opt['Y'] = Y0
    system_opt['N'] = N0
    
    f=inte1(data_offline['p_05'],data_offline.index,fill_value=(data_offline.index[0],data_offline.index[-1]),bounds_error=False)
   
    inicio=0
    for tk in range(inicio,nt-horizon):#nt-horizon):

        if bs[tk] == 0:
            U_sol += [0]
            V_sol+= [0]
            x_sol += [xs[tk]]
            y_sol += [ys[tk]]
            system_opt = system_update(system_opt, xs[tk], ys[tk], 0)
            barxt, baryt, privt = privacy(system_opt)
            barx_sol += [barxt] 
            bary_sol += [baryt] 
            priv_sol += [privt]
            # Compute utility
            util_sol += [0]
            time_sol += [0]
            tra_sol +=[0]
            util_sol_b+=[0]
            dife+=[0]
            sol_min+= [-1]
            

        else:
            #minimizing privacy
            time_0= time.time()
            Xf=np.zeros(horizon+1)
            Yf=np.zeros(horizon+1)
            Nf=np.ones(horizon+1)
            
            Xf[0]=xs[tk]
            Yf[0]=ys[tk]
           
            #Computing future privacy en horizon steps 
            system_aux=system_opt.copy()
            system_aux=system_update(system_aux,Xf[0],Yf[0],Nf[0])
            for kk in range(1,horizon+1):
                Xf[kk],Yf[kk],p1=privacy(system_aux.iloc[1:])
                system_aux=system_update(system_aux,Xf[kk],Yf[kk],Nf[kk])
            
            bx,by,p1 =privacy(system_aux)
            
            #Comparing future privacy with bound
            util_bound=f(p_bound-p1)

            util_sol_b+=[util_bound]
            dife+=[p_bound-p1]
            if p1>=p_bound:#not intervention necesarry
                u_opt = 0
                v_opt = 0
                sol_min+= [-2] 
            
            #mode=1 => MPC horizon=1
            #mode=0 => Using 2 canditates
            elif mode==1:
                X0 = system_opt['X'].tolist()
                Y0 = system_opt['Y'].tolist()
                N0 = system_opt['N'].tolist()

                # Initial conditions
                Jk = 0
                w=[]
                w0 = []
                lbw = []
                ubw = []
                g=[]
                lbg = []
                ubg = []

                # Constraints on input
                U = MX.sym("U",1)
                V = MX.sym("V",1)
                w += [U, V]
                lbw += [-inf]*2
                ubw += [inf]*2
                w0 += [0]*2
                g   += [U**2 + V**2]
                lbg += [0]
                ubg += [util_bound**2]

                # Initial conditions
                Xk = MX.sym("Xk",nbuf)
                w += [Xk]
                lbw += X0
                ubw += X0
                w0 += X0
                Yk = MX.sym("Yk",nbuf)
                w += [Yk]
                lbw += Y0
                ubw += Y0
                w0 += Y0
                Nk = MX.sym("Nk",nbuf)
                w += [Nk]
                lbw += N0
                ubw += N0
                w0 += N0

                Xkfut = MX.sym("Xkfut",1) 
                w += [Xkfut]
                lbw += [Xf[0]] 
                ubw += [Xf[0]] 
                w0 += [Xf[0]] 
                
                Ykfut = MX.sym("Ykfut",1) 
                w += [Ykfut]
                lbw += [Yf[0]]
                ubw += [Yf[0]]
                w0 += [Yf[0]]

                Nkfut = MX.sym("Nkfut",1) 
                w += [Nkfut]
                lbw += [Nf[0]]
                ubw += [Nf[0]]
                w0 += [Nf[0]]

                # Apply the first input 
                Xk = F(Xk,Xkfut[0]+U)
                Yk = F(Yk,Ykfut[0]+V)
                Nk = F(Nk,1)
                Jt = J(x=Xk,y=Yk,nx=Nk)
                Jk += Jt['P']
                
                prob = {'f': -Jk, 'x': vertcat(*w), 'g': vertcat(*g)}
                solver = nlpsol('solver', 'ipopt', prob);
            
                sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
                w_opt = sol['x'].full().flatten()
                J_opt = -sol['f']
                # Plot the solution
                u_opt = w_opt[0]
                v_opt = w_opt[1]
                sol_min+= [-1]
            elif mode==0:
                system_aux2=system_opt.copy()
                xc,yc,p1=privacy(system_aux2.iloc[1:])
                
                candidates=np.zeros((2,2))
                candidates[0,0]=util_bound*(xc-xs[tk])/np.sqrt((xc-xs[tk])**2+(yc-ys[tk])**2)
                candidates[1,0]=util_bound*(yc-ys[tk])/np.sqrt((xc-xs[tk])**2+(yc-ys[tk])**2)
                
                candidates[0,1]=-util_bound*(xc-xs[tk])/np.sqrt((xc-xs[tk])**2+(yc-ys[tk])**2)
                candidates[1,1]=-util_bound*(yc-ys[tk])/np.sqrt((xc-xs[tk])**2+(yc-ys[tk])**2)
                
                candidates_fval=np.zeros(2)
                for i in range(2):
                    system_aux2=system_update(system_opt.copy(),Xf[0]+candidates[0,i],Yf[0]+candidates[1,i],1)
                    b1,b2,candidates_fval[i]=privacy(system_aux2)
                 
                i_sol=np.argmax(candidates_fval)
                u_opt=candidates[0,i_sol]
                v_opt=candidates[1,i_sol]
                sol_min+= [i_sol]
                
            else:
                deg=np.linspace(0,2*np.pi,72,endpoint=False)
                #deg=np.random.uniform(0,2*np.pi,100)
                candidates_fval=np.zeros_like(deg)
                for i in range(len(deg)):
                    system_aux2=system_update(system_opt.copy(),Xf[0]+np.cos(deg[i])*util_bound,Yf[0]+np.sin(deg[i])*util_bound,1)
                    b1,b2,candidates_fval[i]=privacy(system_aux2)
                i_sol=np.argmax(candidates_fval)
                u_opt=np.cos(deg[i_sol])*util_bound
                v_opt=np.sin(deg[i_sol])*util_bound
                sol_min+= [i_sol]
                    
                
                
            U_sol += [u_opt]
            V_sol += [v_opt]
            x_sol += [Xf[0] + u_opt]
            y_sol += [Yf[0] + v_opt]
            system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
            barxt, baryt, privt = privacy(system_opt)
            barx_sol += [barxt] 
            bary_sol += [baryt] 
            priv_sol += [privt] 
            # Compute utility
            util_sol += [utility(u_opt,v_opt)]
            time_sol += [time.time()-time_0]
            tra_sol +=[1]
            
    clear_output(wait=False)

    data_sol = pd.DataFrame()
    data_sol['x'] = x_sol
    data_sol['y'] = y_sol
    data_sol['u_enu']= us[:len(x_sol)]
    data_sol['U'] = U_sol
    data_sol['V'] = V_sol
    data_sol['barx'] = barx_sol
    data_sol['bary'] = bary_sol
    data_sol['priv'] = priv_sol
    #data_mpc['priv_hp'] = priv_hp
    #data_mpc['priv_mpc_hp'] = priv_mpc_hp
    data_sol['util'] = util_sol
    data_sol['time'] = time_sol
    data_sol['tra'] =tra_sol
    data_sol['util_sol_b']=util_sol_b
    data_sol['dife']=dife
    data_sol['indice_sol']=sol_min
    
    return data_sol#U_mpc, V_mpc, x_mpc, y_mpc, barx_mpc, bary_mpc, priv_mpc, util_mpc 

## Solving online instance

In [10]:
#data_offline=pd.read_csv(sol_file+'/gains/gain_'+str(Hp)+'.csv',delimiter=',',index_col='utility')
#data_offline_st=pd.read_csv(sol_file+'/gain_st.csv',delimiter=',',index_col='utility')
p_bound=1000000
#mode=1
Data_sol = {} # Empty dictionary
#Data_sol_speed = {}

data_offline=pd.read_csv(sol_file+'/gains/gain_'+str(Hp)+'.csv',delimiter=',',index_col='utility')
#start_time = time.monotonic()
delta_time_mode=[]
for mode in [0,1,2]:
    start_time = time.monotonic()
    Data_sol[mode]= solve_method(xs,ys,bs,vs,Hp,nbuf,p_bound,data_offline,mode)# Empty dictionary
    end_time = time.monotonic()
    delta_time_mode+=[timedelta(seconds=end_time - start_time)]
    print(timedelta(seconds=end_time - start_time))
    

print(delta_time_mode)



0:01:03.578000
[datetime.timedelta(seconds=10, microseconds=625000), datetime.timedelta(seconds=27, microseconds=297000), datetime.timedelta(seconds=63, microseconds=578000)]


In [11]:
p_real=Data_real.loc[Data_real.b>0].priv.mean()
p_GeoI=Data_obf.loc[Data_real.b>0].priv.mean()
p_0=Data_sol[0].loc[Data_sol[0].tra>0].priv.mean()
p_1=Data_sol[1].loc[Data_sol[1].tra>0].priv.mean()
p_2=Data_sol[2].loc[Data_sol[2].tra>0].priv.mean()
util_sol0=Data_sol[0].loc[Data_sol[0].tra>0].util.mean()
util_sol1=Data_sol[1].loc[Data_sol[1].tra>0].util.mean()
util_sol2=Data_sol[2].loc[Data_sol[2].tra>0].util.mean()
#p_0_sp=Data_sol_speed[0].loc[Data_sol_speed[0].tra>0].priv.mean()
#p_1_sp=Data_sol_speed[1].loc[Data_sol_speed[1].tra>0].priv.mean()
#util_sol_sp=Data_sol_speed[0].loc[Data_sol_speed[0].tra>0].util.mean()


print("Privacy real: ",p_real )
print("Privacy Geo-I: ",p_GeoI )
print("Privacy mode 0      : ",p_0," ",(p_0-p_real)/p_real)
#print("Privacy mode 0 speed: ",p_0_sp," ",(p_0_sp-p_real)/p_real)
print("Privacy mode 1      : ",p_1," ",(p_1-p_real)/p_real)
#print("Privacy mode 1 speed: ",p_1_sp," ",(p_1_sp-p_real)/p_real)
print("Privacy mode 2      : ",p_2," ",(p_2-p_real)/p_real,"\n")
print("mean utility mode 0:",util_sol0)
print("mean utility mode 1:",util_sol1)
print("mean utility mode 2:",util_sol2,"\n")

print("resolution average time mode 0:",Data_sol[0].loc[Data_sol[0].tra>0].time.mean(),'s')
print("resolution average time mode 1:",Data_sol[1].loc[Data_sol[1].tra>0].time.mean(),'s')
print("resolution average time mode 2:",Data_sol[2].loc[Data_sol[2].tra>0].time.mean(),'s')
#print("mean utility speed",util_sol_sp)



Privacy real:  5815745.097692999
Privacy Geo-I:  6303712.008531598
Privacy mode 0      :  6472588.404842224   0.11294224490852992
Privacy mode 1      :  6475557.893468882   0.11345283960908444
Privacy mode 2      :  6465300.529944269   0.1116891165860995 

mean utility mode 0: 475.9541577980965
mean utility mode 1: 476.4494015774833
mean utility mode 2: 478.74697581194624 

resolution average time mode 0: 0.0628954569498698 s
resolution average time mode 1: 0.1772297869126002 s
resolution average time mode 2: 0.4293093929688136 s


## Ploting solutions

In [12]:
#Comparing solution with mode 0,1 and 2.

from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d
inicio=0
Timep2=save_data['Time'][inicio:len(xs)-Hp].loc[save_data['bs']>0]

fig = figure(plot_width=900, plot_height=400)

fig.line(Timep, Data_real.loc[Data_real.b>0].priv, line_color='black', legend_label="priv real")
fig.circle(Timep, Data_real.loc[Data_real.b>0].priv, color='black', legend_label="priv real")
colors=['red','green','violet','chocolate']
for mode in [0,1,2]:
    fig.line(Timep2, Data_sol[mode].loc[Data_sol[mode].tra>0].priv, line_color=colors[mode], legend_label="priv mode "+str(mode))
    fig.circle(Timep2, Data_sol[mode].loc[Data_sol[mode].tra>0].priv, color=colors[mode], legend_label="priv mode "+str(mode))
    #fig.line(Timep2, Data_sol_speed[mode].loc[Data_sol_speed[mode].tra>0].priv, line_color=colors[mode+2], legend_label="priv mode "+str(mode)+'_speed')
    #fig.circle(Timep2, Data_sol_speed[mode].loc[Data_sol_speed[mode].tra>0].priv, color=colors[mode+2], legend_label="priv mode "+str(mode)+'_speed')
fig.line(Timep2, p_bound*np.ones(len(Timep2)), line_color='orange', legend_label="limit "+str(p_bound),line_dash=[6,3])
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "privacy"
show(fig)

fig = figure(plot_width=900, plot_height=250)
fig.circle(Timep2,Data_sol[0].loc[Data_sol[0].tra>0].util_sol_b,color='red',legend_label='mode 0')
fig.circle(Timep2,Data_sol[1].loc[Data_sol[1].tra>0].util_sol_b,color='green',legend_label='mode 1')
fig.circle(Timep2,Data_sol[2].loc[Data_sol[2].tra>0].util_sol_b,color='violet',legend_label='mode 2')
fig.line(Timep2,Data_sol[0].loc[Data_sol[0].tra>0].util_sol_b,color='red',legend_label='mode 0')
fig.line(Timep2,Data_sol[1].loc[Data_sol[1].tra>0].util_sol_b,color='green',legend_label='mode 1')
fig.line(Timep2,Data_sol[2].loc[Data_sol[2].tra>0].util_sol_b,color='violet',legend_label='mode 2')
#fig.circle(Timep2,Data_sol_speed[0].loc[Data_sol_speed[0].tra>0].util_sol_b,color='violet',legend_label='mode 0_speed')
#fig.circle(Timep2,Data_sol_speed[1].loc[Data_sol_speed[1].tra>0].util_sol_b,color='chocolate',legend_label='mode 1_speed')
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "utility"
show(fig)

fig = figure(plot_width=900, plot_height=500)
fig.circle(Timep2,Data_sol[0].loc[Data_sol[0].tra>0].U,color='red',legend_label='mode 0')
fig.circle(Timep2,Data_sol[1].loc[Data_sol[1].tra>0].U,color='green',legend_label='mode 1')
fig.circle(Timep2,Data_sol[2].loc[Data_sol[2].tra>0].U,color='violet',legend_label='mode 2')
#fig.circle(Timep2,Data_sol_speed[0].loc[Data_sol_speed[0].tra>0].U,color='violet',legend_label='mode 0_speed')
#fig.circle(Timep2,Data_sol_speed[1].loc[Data_sol_speed[1].tra>0].U,color='chocolate',legend_label='mode 1_speed')
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "Delta x"
show(fig)

fig = figure(plot_width=900, plot_height=250)
fig.circle(Timep2,Data_sol[0].loc[Data_sol[0].tra>0].V,color='red',legend_label='mode 0')
fig.circle(Timep2,Data_sol[1].loc[Data_sol[1].tra>0].V,color='green',legend_label='mode 1')
fig.circle(Timep2,Data_sol[2].loc[Data_sol[2].tra>0].V,color='violet',legend_label='mode 2')
#fig.circle(Timep2,Data_sol_speed[0].loc[Data_sol_speed[0].tra>0].V,color='violet',legend_label='mode 0_speed')
#fig.circle(Timep2,Data_sol_speed[1].loc[Data_sol_speed[1].tra>0].V,color='chocolate',legend_label='mode 1_speed')
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "Delta y"
show(fig)




In [13]:
fig = figure(plot_width=900, plot_height=500)
fig.line(Data_real.loc[Data_real.b>0].x, Data_real.loc[Data_real.b>0].y, line_color='black', legend_label="priv real")
fig.circle(Data_real.loc[Data_real.b>0].x, Data_real.loc[Data_real.b>0].y, color='black', legend_label="priv real")
colors=['red','green','violet','chocolate']
for mode in [0,1,2]:
    fig.line(Data_sol[mode].loc[Data_sol[mode].tra>0].x, Data_sol[mode].loc[Data_sol[mode].tra>0].y, line_color=colors[mode], legend_label="priv mode "+str(mode))
    fig.circle(Data_sol[mode].loc[Data_sol[mode].tra>0].x, Data_sol[mode].loc[Data_sol[mode].tra>0].y, color=colors[mode], legend_label="priv mode "+str(mode))
    #fig.line(Data_sol[mode].loc[Data_sol_speed[mode].tra>0].x, Data_sol_speed[mode].loc[Data_sol[mode].tra>0].y, line_color=colors[mode+2], legend_label="priv mode "+str(mode)+'_speed')
    #fig.circle(Data_sol[mode].loc[Data_sol_speed[mode].tra>0].x, Data_sol_speed[mode].loc[Data_sol[mode].tra>0].y, color=colors[mode+2], legend_label="priv mode "+str(mode)+'_speed')
fig.xaxis.axis_label = "x"
fig.yaxis.axis_label = "y"
show(fig)

## Comparing with MPC


In [14]:
def solve_mpc(xs,ys,bs,horizon,nbuf,util_bound):
    nt = len(xs)
    x_mpc = []
    y_mpc = []
    U_mpc = []
    V_mpc = []
    barx_mpc = []
    bary_mpc = []
    priv_mpc = []
    util_mpc = []
    time_mpc = []
    X0 = np.array([0]*nbuf).tolist()
    Y0 = np.array([0]*nbuf).tolist()
    N0 = np.array([0]*nbuf).tolist()

    system_opt = pd.DataFrame()
    system_opt['X'] = X0
    system_opt['Y'] = Y0
    system_opt['N'] = N0

    for tk in range(0,nt-horizon):#nt-horizon):

        if bs[tk] == 0:
            U_mpc += [0]
            V_mpc += [0]
            x_mpc += [xs[tk]]
            y_mpc += [ys[tk]]
            system_opt = system_update(system_opt, xs[tk], ys[tk], 0)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [0]
            time_mpc += [0]
            #system_opt = system_update(system_opt, xs[tk], ys[tk], 0)

        else:
            time_0= time.time()
            X0 = system_opt['X'].tolist()
            Y0 = system_opt['Y'].tolist()
            N0 = system_opt['N'].tolist()

            # Initial conditions
            Jk = 0
            w=[]
            w0 = []
            lbw = []
            ubw = []
            g=[]
            lbg = []
            ubg = []

            # Constraints on input
            U = MX.sym("U",1)
            V = MX.sym("V",1)
            w += [U, V]
            lbw += [-inf]*2
            ubw += [inf]*2
            w0 += [0]*2
            g   += [U**2 + V**2]
            lbg += [0]
            ubg += [util_bound[tk]**2]

            # Initial conditions
            Xk = MX.sym("Xk",nbuf)
            w += [Xk]
            lbw += X0
            ubw += X0
            w0 += X0
            Yk = MX.sym("Yk",nbuf)
            w += [Yk]
            lbw += Y0
            ubw += Y0
            w0 += Y0
            Nk = MX.sym("Nk",nbuf)
            w += [Nk]
            lbw += N0
            ubw += N0
            w0 += N0

            # Future values of x, y and n
            Xf = xs[tk:tk+horizon]
            Xf = Xf[::].tolist()
            Xkfut = MX.sym("Xkfut",horizon) 
            w += [Xkfut]
            lbw += Xf
            ubw += Xf
            w0 += Xf
            Yf = ys[tk:tk+horizon]
            Yf = Yf[::].tolist()
            Ykfut = MX.sym("Ykfut",horizon) 
            w += [Ykfut]
            lbw += Yf
            ubw += Yf
            w0 += Yf
            Nf = bs[tk:tk+horizon]
            Nf = Nf[::].tolist()
            Nkfut = MX.sym("Nkfut",horizon) 
            w += [Nkfut]
            lbw += Nf
            ubw += Nf
            w0 += Nf

            # Apply the first input 
            Xk = F(Xk,Xkfut[0]+U)
            Yk = F(Yk,Ykfut[0]+V)
            Nk = F(Nk,1)
            Jt = J(x=Xk,y=Yk,nx=Nk)
            Jk += Jt['P']
            for k in range(horizon-1):
                # New NLP variable for the control
                Uk = MX.sym('U_' + str(k))
                Vk = MX.sym('V_' + str(k))
                w   += [Uk, Vk]
                lbw += [-util_bound[tk+k+1]*Nf[k+1]]*2
                ubw += [util_bound[tk+k+1]*Nf[k+1]]*2
                w0 += [0]*2

                g   += [Uk**2 + Vk**2]
                lbg += [0]
                ubg += [util_bound[tk+k+1]**2]

                Xk = F(Xk,Xkfut[k+1]+Uk)
                Yk = F(Yk,Ykfut[k+1]+Vk)
                Nk = F(Nk,Nkfut[k+1])
                Jt = J(x=Xk,y=Yk,nx=Nk)
                Jk += Jt['P']


            prob = {'f': -Jk, 'x': vertcat(*w), 'g': vertcat(*g)}
            solver = nlpsol('solver', 'ipopt', prob);
            sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
            w_opt = sol['x'].full().flatten()
            J_opt = -sol['f']

            # Plot the solution
            u_opt = w_opt[0]
            v_opt = w_opt[1]
            U_mpc += [u_opt]
            V_mpc += [v_opt]
            x_mpc += [Xf[0] + u_opt]
            y_mpc += [Yf[0] + v_opt]
            system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [utility(u_opt,v_opt)]
            time_mpc += [time.time()-time_0]
            #system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
   
    clear_output(wait=False)

    data_mpc = pd.DataFrame()
    data_mpc['x'] = x_mpc
    data_mpc['y'] = y_mpc
    data_mpc['u_enu']= us[:len(x_mpc)]
    data_mpc['U'] = U_mpc
    data_mpc['V'] = V_mpc
    data_mpc['barx'] = barx_mpc
    data_mpc['bary'] = bary_mpc
    data_mpc['priv'] = priv_mpc
    data_mpc['util'] = util_mpc
    data_mpc['time'] = time_mpc
    
    return data_mpc#U_mpc, V_mpc, x_mpc, y_mpc, barx_mpc, bary_mpc, priv_mpc, util_mpc 


In [15]:
#data_mpc=solve_mpc(xs,ys,bs,1,nbuf,util_sol*np.ones(len(xs)))

data_mpc=solve_mpc(xs[:-Hp],ys[:-Hp],bs[:-Hp],1,nbuf,Data_sol[0].util)


In [16]:
fig = figure(plot_width=900, plot_height=250)
fig.line(Timep, Data_real.loc[Data_real.b>0].priv, line_color='black', legend_label="priv real")
fig.circle(Timep, Data_real.loc[Data_real.b>0].priv, color='black', legend_label="priv real")
colors=['orange','green','red','yellow']
for mode in [0]:
    fig.line(Timep2, Data_sol[mode].loc[Data_sol[mode].tra>0].priv, line_color='red', legend_label="priv mode "+str(mode))
    fig.circle(Timep2, Data_sol[mode].loc[Data_sol[mode].tra>0].priv, color='red', legend_label="priv mode "+str(mode))
    

fig.line(Timep2, data_mpc.loc[data_mpc.util>0].priv, line_color='blue', legend_label="priv mpc")
fig.circle(Timep2, data_mpc.loc[data_mpc.util>0].priv, color='blue', legend_label="priv mpc")
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "privacy"
show(fig)

fig = figure(plot_width=900, plot_height=500)
fig.line(Data_real.loc[Data_real.b>0].x, Data_real.loc[Data_real.b>0].y, line_color='black', legend_label="priv real")
fig.circle(Data_real.loc[Data_real.b>0].x, Data_real.loc[Data_real.b>0].y, color='black', legend_label="priv real")

colors=['orange','green','red','yellow']
for mode in [2]:
    fig.line(Data_sol[mode].loc[Data_sol[mode].tra>0].x, Data_sol[mode].loc[Data_sol[mode].tra>0].y, line_color='red', legend_label="priv mode "+str(mode))
    fig.circle(Data_sol[mode].loc[Data_sol[mode].tra>0].x, Data_sol[mode].loc[Data_sol[mode].tra>0].y, color='red', legend_label="priv mode "+str(mode))
    

fig.line(data_mpc.loc[data_mpc.util>0].x, data_mpc.loc[data_mpc.util>0].y, line_color='blue', legend_label="priv mpc")
fig.circle(data_mpc.loc[data_mpc.util>0].x, data_mpc.loc[data_mpc.util>0].y, color='blue', legend_label="priv mpc")
fig.xaxis.axis_label = "x"
fig.yaxis.axis_label = "y"
show(fig)


In [17]:
p_mpc=data_mpc.loc[data_mpc.util>0].priv.mean()
print("Privacy real: ",p_real )
print("Privacy mode 0: ",p_0," ",(p_0-p_real)/p_real)
print("Privacy mode 1: ",p_1," ",(p_1-p_real)/p_real)
print("Privacy mode 2: ",p_2," ",(p_2-p_real)/p_real)
print("Privacy mpc: ",p_mpc," ",(p_mpc-p_real)/p_real)
print("mean utility",util_sol2,'\n')

print("resolution average time mode   0:",Data_sol[0].loc[Data_sol[0].tra>0].time.mean(),'s')
print("resolution average time mode   1:",Data_sol[1].loc[Data_sol[1].tra>0].time.mean(),'s')
print("resolution average time mode   2:",Data_sol[2].loc[Data_sol[2].tra>0].time.mean(),'s')
print("resolution average time mpc     :",data_mpc.loc[data_mpc.util>0].time.mean(),'s')

Privacy real:  5815745.097692999
Privacy mode 0:  6472588.404842224   0.11294224490852992
Privacy mode 1:  6475557.893468882   0.11345283960908444
Privacy mode 2:  6465300.529944269   0.1116891165860995
Privacy mpc:  6440133.484630527   0.10736171830935498
mean utility 478.74697581194624 

resolution average time mode   0: 0.0628954569498698 s
resolution average time mode   1: 0.1772297869126002 s
resolution average time mode   2: 0.4293093929688136 s
resolution average time mpc     : 0.40365173584885067 s


## Ploting with respect to p_bound

In [18]:
data_offline=pd.read_csv(sol_file+'/gains/gain_'+str(Hp)+'.csv',delimiter=',',index_col='utility')

sol_newm=sol_file+'/sol_online'
if not os.path.exists(sol_newm):
    os.makedirs(sol_newm)

p_bound=1000000
mode=0
Data_sol_p = {} # Solution worst case method
Data_sol_mpc = {} # Solution with mpc previous work
start_time = time.monotonic()
#lower_bounds=np.arange(500,2001,100)#cabspotting
lower_bounds=np.arange(500000,2000001,250000)
np.savetxt(sol_newm+'/lb.txt',lower_bounds)

for p_lb in lower_bounds:
    print('lower bound: ',p_lb)
    Data_sol_p[p_lb]= solve_method(xs,ys,bs,vs,Hp,nbuf,p_lb,data_offline,mode)# Empty dictionary
    end_time = time.monotonic()
    Data_sol_p[p_lb].to_csv(sol_newm+'/sol_h_'+str(Hp)+'_lb_'+str(int(p_lb))+'_mode'+str(mode)+'.csv')
    util_sol=Data_sol_p[p_lb].loc[Data_sol_p[p_lb].tra>0].util.mean()
    
    Data_sol_mpc[p_lb]=solve_mpc(xs[:-Hp],ys[:-Hp],bs[:-Hp],1,nbuf,Data_sol_p[p_lb].util)
    Data_sol_mpc[p_lb].to_csv(sol_newm+'/sol_h_'+str(Hp)+'_lb_'+str(int(p_lb))+'_mpc.csv')
    


In [19]:
print("Privacy real: ",p_real )
plot_pr_b=[]
plot_pr_b_perc=[]
plot_pr_b_speed=[]
plot_pr_b_perc_speed=[]
plot_pr_m2=[]
plot_pr_m2_perc=[]
plot_pr_mpc=[]
plot_pr_mpc_perc=[]


for p_lb in lower_bounds:
    #load privacy
    p_b      =Data_sol_p[p_lb].loc[Data_sol_p[p_lb].tra>0].priv.mean() #comparing just transmitted points
    p_mpc    =Data_sol_mpc[p_lb].loc[Data_sol_mpc[p_lb].util>0].priv.mean()#comparing just transmitted points
    
    #compute valors
    plot_pr_b+=[p_b]
    plot_pr_b_perc+=[(p_b-p_real)/p_real]
    util_sol=Data_sol_p[p_lb].loc[Data_sol_p[p_lb].tra>0].util.mean()
    
    #plot_pr_b_speed+=[p_b_speed]
    #plot_pr_b_perc_speed+=[(p_b_speed-p_real)/p_real]
    #util_sol_speed=Data_sol_p_speed[p_lb].loc[Data_sol_p_speed[p_lb].tra>0].util.mean()
  
    plot_pr_mpc+=[p_mpc]
    plot_pr_mpc_perc+=[(p_mpc-p_real)/p_real]
    
    print("Privacy bound "+str(p_lb)+"      : ",p_b," ",(p_b-p_real)/p_real,' utility mean ', util_sol)
    print("Privacy mpc            : ",p_mpc," ",(p_mpc-p_real)/p_real,'\n')
    
    print("resolution average time    :",Data_sol_p[p_lb].loc[Data_sol_p[p_lb].tra>0].time.mean(),'s')
    print("resolution average time mpc:",Data_sol_mpc[p_lb].loc[Data_sol_mpc[p_lb].util>0].time.mean(),'s\n')
    
    
    
    

Privacy real:  5815745.097692999
Privacy bound 500000      :  6136941.603236023   0.05522877982916361  utility mean  254.03712187158484
Privacy mpc            :  6120278.94315365   0.05236368519339914 

resolution average time    : 0.05405266914102766 s
resolution average time mpc: 0.4758819225761626 s

Privacy bound 750000      :  6281515.14868928   0.08008776918043464  utility mean  366.4062921361461
Privacy mpc            :  6257110.74980692   0.0758915056798144 

resolution average time    : 0.06269299818409814 s
resolution average time mpc: 0.44301528731981915 s

Privacy bound 1000000      :  6472588.404842224   0.11294224490852992  utility mean  475.9541577980965
Privacy mpc            :  6440133.484630527   0.10736171830935498 

resolution average time    : 0.0645217713382509 s
resolution average time mpc: 0.39729995528856915 s

Privacy bound 1250000      :  6678294.851298144   0.14831285400512187  utility mean  565.460577010999
Privacy mpc            :  6644491.657759263   0.14

In [20]:
# Ploting privacy curve 

fig = figure(plot_width=900, plot_height=300)
fig.line(Timep, Data_real.loc[Data_real.b>0].priv, line_color='black', legend_label="priv real")
fig.circle(Timep, Data_real.loc[Data_real.b>0].priv, color='black', legend_label="priv real")
forlb=1500000

for lb in [forlb]:
    fig.line(Timep2, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].priv, line_color='red', legend_label="priv wc method "+str(lb))
    fig.circle(Timep2, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].priv, color='red', legend_label="priv wc method "+str(lb))  

    
    fig.line(Timep2, Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].priv, line_color='blue', legend_label="priv mpc",alpha=0.5)
    fig.circle(Timep2, Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].priv, color='blue', legend_label="priv mpc",alpha=0.25)
    
    fig.line(Timep2, lb*np.ones(len(Timep2)), line_color='red', legend_label="limit "+str(lb),line_dash=[6,3])
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "privacy"
fig.y_range = Range1d(0, forlb*2)
show(fig)


#Utility
fig = figure(plot_width=900, plot_height=300)
for lb in [forlb]:
    fig.line(Timep2, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].util, line_color='red', legend_label="utility loss wc method "+str(lb))
    fig.circle(Timep2, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].util, color='red', legend_label="utility loss wc method "+str(lb))
    
show(fig)
    
#Position
fig = figure(plot_width=900, plot_height=500)
fig.line(Data_real.loc[Data_real.b>0].x, Data_real.loc[Data_real.b>0].y, line_color='black', legend_label="actual pos")
fig.circle(Data_real.loc[Data_real.b>0].x, Data_real.loc[Data_real.b>0].y, color='black', legend_label="actual pos")
for lb in [forlb]:
    fig.line(Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].x, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].y, line_color='red', legend_label="wc method")
    fig.circle(Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].x, Data_sol_p[lb].loc[Data_sol_p[lb].tra>0].y, color='red', legend_label="wc method")
    
    fig.line(Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].x, Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].y, line_color='blue', legend_label="mpc",alpha=0.5)
    fig.circle(Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].x, Data_sol_mpc[lb].loc[Data_sol_mpc[lb].util>0].y, color='blue', legend_label="mpc",alpha=0.25)
fig.xaxis.axis_label = "x"
fig.yaxis.axis_label = "y"
show(fig)
